In [1]:
#updated == 2
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.prompts import PromptTemplate
from langchain.llms.bedrock import Bedrock
from langchain.chains import LLMChain
import boto3
import botocore

config = botocore.config.Config(
    read_timeout=900,
    connect_timeout=900,
    retries={"max_attempts": 3}
)

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=config,
)
path = (r"C:\Users\Lenovo\Documents\Project-vs code\Amazon Transcribe\One Pharma\517792954-SBI-Card-Statement-0824-22-07-2021.pdf")
loader = PDFPlumberLoader(path)
docs = loader.load()

In [2]:
template = """
    Extract the text from the bank transcation statement and print the results in the JSON format only

    for example:(make below JSON format as the reference and print exactly like the below format)
        
    "user_id":
    "statement_id":
    "identity": 
    "bank":
    "credit_card_number":
    "name":
    "address":
    "payment_due_date":
    "total_dues":
    "min_amt_due":
    "finance_charges":
    "credit_limit":
    "avl_credit_limit":
    "cash_limit":
    "avl_cash_limit":
    "opening_balance":
    "payment_or_credits":
    "purchase_or_debits":
    "statement_date": 
    "points_earned" : 
    "total_points": 

        "transactions":
            
                "amount": 
                "transaction_type":
                "date": 
                "transaction_narration":
                "transaction_details": 
                "transaction_ref_number":

the above all the details are to be in JSON

        Instruction and Description for the output format:
            
                statement_id: Unique statement id for each statement to be extracted
                user_id: Unique id for the user (if not present means simpley leave it blank)
                identity: Dictionary of user details present in the statement
                bank: Name of the bank
                credit_card_number: Credit Card number
                name: Customer name
                address: Address of the customer
                payment_due_date: Due date of the credit card bill
                total_dues: Total Amount Due
                min_amt_due: Minimum Due Amount
                finance_charges: Amount of interest charged on the amount of money borrowed
                cash_limit: take datas from Cash Limit
                credit_limit: Credit Limit including cash(take from 'Credit Limit')
                avl_credit_limit: Available credit limit (take datas only from 'Available Credit Limit' in the document)
                avl_cash_limit: Available cash limit (take datas only from 'Available Cash Limit' in the document)
                opening_balance: take datas only from 'Previous Balance' in the document
                payment_or_credits: take datas only from 'Payments, Reversals & other Credits' in the document
                purchase_or_debits: Total of the purchase and debits (take datas only from 'Purchases & Other Debits' in the document)
                statement_date: Date of the statement
                points_earned: Points earned in the statement cycle (take from 'Earned' in the document)
                total_points: Total points earned till now(take datas from 'Closing Balance' in the document)
                transactions: List of transactions
                amount: Transaction Amount
                transaction_type: Type of transaction (mention like Credit or Debit ,don't mention other than this)
                transaction_narration: The transaction narration present in the statement
                transaction_details: Any other details of the transaction if available
                transaction_ref_number: Transaction reference number if available

        - all the dates must be print in the formart like yyyy-MM-dd
                for example:
                       if the date is like "09 Jul 21" means it should print like "2021-07-09" 
                so format and print the date like this format only for all the datas 

so the i need to extract all the transcations and convert into JSON and print the accurate results for all the transcations for given document
        
        Instruction:
               - Convert all transaction data into JSON format, ensuring complete extraction and conversion without any omissions or data mismatches throughout the entire transaction process.
               - Must convert all the datas as JSON in the given text{datas} 
               - Give for all the transcations
               - transaction_type (strictly mention like Credit or Debit ,don't mention other than this)
               - For empty fields, must be "None"
{datas}
"""

In [4]:
qa_prompt = PromptTemplate(template=template, input_variables=["datas"])
llm = Bedrock(model_id="anthropic.claude-v2:1",client=bedrock_client,model_kwargs = {"temperature":1e-10,"max_tokens_to_sample": 40000})
llm_chain = LLMChain(prompt=qa_prompt, llm=llm, verbose= False)
result = llm_chain.run(datas= docs)

In [6]:
import re

# Provided text containing JSON
provided_text =result

# Extract JSON portion
start_index = provided_text.find('{')
end_index = provided_text.rfind('}') + 1
json_data = provided_text[start_index:end_index]

# Print the extracted JSON
print(json_data)


{
  "user_id": "None",
  "statement_id": "B21073266361",
  "identity": {
    "name": "SUNIL KUMAR YADAV"
  },
  "bank": "SBI Card",  
  "credit_card_number": "XXXXXXXXXX24",
  "name": "SUNIL KUMAR YADAV",
  "address": "None",
  "payment_due_date": "2021-08-11",
  "total_dues": 33051,
  "min_amt_due": 5717, 
  "finance_charges": 2327.62,
  "credit_limit": 200000,
  "avl_credit_limit": 158903.54,
  "cash_limit": 60000,
  "avl_cash_limit": 60000,
  "opening_balance": 13396.71,
  "payment_or_credits": 13778.47,
  "purchase_or_debits": 39150.6,
  "statement_date": "2021-07-22",
  "points_earned": 2706,
  "total_points": 5037,
  "transactions": [
    {
      "amount": 39,
      "transaction_type": "Credit",
      "date": "2021-07-09",
      "transaction_narration": "PETROL TRXN FEE RVRSL EXCLUDING TAX",
      "transaction_details": "None",
      "transaction_ref_number": "None"
    },
    {
      "amount": 950,
      "transaction_type": "Debit", 
      "date": "2021-07-15",
      "transactio